In [133]:
import os
import torch
import pytest
from pprint import pprint
import numpy as np

from conditional_rate_matching.models.generative_models.crm import CRM
from conditional_rate_matching.data.music_dataloaders import LankhPianoRollDataloader
from conditional_rate_matching.data.music_dataloaders_config import LakhPianoRollConfig
from conditional_rate_matching.models.temporal_networks.temporal_networks_config import SequenceTransformerConfig
from conditional_rate_matching.models.temporal_networks.temporal_networks_utils import load_temporal_network
from conditional_rate_matching.configs.experiments_configs.crm.crm_experiments_music import experiment_music_conditional_config

In [11]:
bridge_conditional = False
config = experiment_music_conditional_config(bridge_conditional=bridge_conditional)
config.temporal_network = SequenceTransformerConfig(num_layers=1,
                                                    d_model=128,
                                                    num_heads=1)

crm = CRM(config)

databatch = next(crm.dataloader_0.train().__iter__())
x = databatch[0]
batch_size = x.size(0)
ts = torch.rand((batch_size,))
print(x.shape)

logits = crm.forward_rate.temporal_network(x,ts)
print(logits.shape)

generative_sample,original_sample = crm.pipeline(32,origin=True)
print(generative_sample.shape)

torch.Size([64, 256])
torch.Size([64, 256, 129])


5it [00:01,  3.89it/s]


torch.Size([32, 256])


In [141]:
def outlier_per_song(generated_song_notes,real_song_notes):
    notes_in_generated_song = set(generated_song_notes.detach().numpy())
    notes_in_real_song = set(real_song_notes.detach().numpy())
    
    notes_in_generated_not_in_real = notes_in_generated_song - notes_in_real_song
    number_of_notes_not = len(notes_in_generated_not_in_real)
    
    total_proportion = number_of_notes_not/config.data1.vocab_size
    
    
    tensor_notes_in_real = torch.Tensor(list(notes_in_real_song))
    # Create a boolean mask for each element in tensor_a indicating whether it's in tensor_b
    mask = torch.isin(generated_song_notes, tensor_notes_in_real)
    
    # Count the number of True values in the mask
    count_of_notes_in_real = torch.sum(mask).item()
    count_not_in_real = real_song_notes.size(0) - count_of_notes_in_real
    
    outliers_per_song = count_not_in_real/real_song_notes.size(0)
    
    return total_proportion,outliers_per_song
    
def obtain_numbers_not_in_real(a,b):
    number_of_songs = generative_sample.size(0)

    total_proportion_list  = []
    outlier_per_song_list = []
    
    for song_index in range(number_of_songs):
        generated_song_notes = generative_sample[song_index]
        real_song_notes = original_sample[song_index]
        proportion,outlier_ = outlier_per_song(generated_song_notes,real_song_notes)
        total_proportion_list.append(proportion)
        outlier_per_song_list.append(outlier_)
    
    total_proportion_list = np.asarray(total_proportion_list)
    outlier_per_song_list = np.asarray(outlier_per_song_list)
    
    total_proportion_mean, total_proportionl_std = total_proportion_list.mean(),total_proportion_list.std()
    outlier_per_song_mean, outlier_per_song_std = outlier_per_song_list.mean(),outlier_per_song_list.std()
    return {total_proportion_mean, total_proportionl_std,outlier_per_song_mean, outlier_per_song_std}

In [111]:
for note in

129

In [117]:
torch.Tensor(list(notes_in_real_song))

tensor([2., 3., 5., 7.])

In [105]:
notes_in_generated_song

{1, 2, 7}

In [125]:
import torch

# Example tensors
tensor_a = torch.tensor([1, 2, 3, 4, 5,2,2,4])  # Tensor to check
tensor_b = torch.tensor([2, 4, 6, 8])    # Tensor to check against

# Flatten tensors if they are not already 1D (optional, depends on your case)
tensor_a_flat = tensor_a.flatten()
tensor_b_flat = tensor_b.flatten()

# Create a boolean mask for each element in tensor_a indicating whether it's in tensor_b
mask = torch.isin(tensor_a_flat, tensor_b_flat)

# Count the number of True values in the mask
count = torch.sum(mask).item()

print(f"Number of elements in tensor_a that are present in tensor_b: {count}")


Number of elements in tensor_a that are present in tensor_b: 5
